## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key= g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('../Weather_Database/WeatherPy_Database.csv')
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ancud,CL,-41.8697,-73.8203,55.99,94,90,16.11,moderate rain
1,1,Hobart,AU,-42.8794,147.3294,48.99,87,14,9.22,few clouds
2,2,Victoria,HK,22.2855,114.1577,77.00,86,100,1.74,overcast clouds
3,3,Mujiayingzi,CN,42.1167,118.7833,40.91,39,100,1.57,overcast clouds
4,4,Maniitsoq,GL,65.4167,-52.9000,35.28,75,10,4.00,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 99


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Victoria,HK,22.2855,114.1577,77.00,86,100,1.74,overcast clouds
8,8,Avera,US,33.1940,-82.5271,80.60,41,1,12.66,clear sky
9,9,Butaritari,KI,3.0707,172.7902,82.35,73,100,10.42,overcast clouds
11,11,Kapaa,US,22.0752,-159.3190,75.20,73,75,13.80,broken clouds
12,12,Mar Del Plata,AR,-38.0023,-57.5575,71.01,60,0,12.66,clear sky
13,13,Carnarvon,AU,-24.8667,113.6333,66.20,59,83,11.50,broken clouds
15,15,San Juan,AR,-31.5375,-68.5364,78.53,28,83,3.53,broken clouds
16,16,Batticaloa,LK,7.7102,81.6924,80.37,82,99,7.31,overcast clouds
20,20,Imphal,IN,24.8167,93.9500,67.50,78,75,2.42,broken clouds
22,22,Ocampo,MX,21.6500,-101.5000,89.01,50,29,17.34,scattered clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


Unnamed: 0             0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

Unnamed: 0             359
City                   359
Country                359
Lat                    359
Lng                    359
Max Temp               359
Humidity               359
Cloudiness             359
Wind Speed             359
Current Description    359
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Victoria,HK,77.00,overcast clouds,22.2855,114.1577,
8,Avera,US,80.60,clear sky,33.1940,-82.5271,
9,Butaritari,KI,82.35,overcast clouds,3.0707,172.7902,
11,Kapaa,US,75.20,broken clouds,22.0752,-159.3190,
12,Mar Del Plata,AR,71.01,clear sky,-38.0023,-57.5575,
13,Carnarvon,AU,66.20,broken clouds,-24.8667,113.6333,
15,San Juan,AR,78.53,broken clouds,-31.5375,-68.5364,
16,Batticaloa,LK,80.37,overcast clouds,7.7102,81.6924,
20,Imphal,IN,67.50,broken clouds,24.8167,93.9500,
22,Ocampo,MX,89.01,scattered clouds,21.6500,-101.5000,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    params["location"] = f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [40]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Victoria,HK,77.00,overcast clouds,22.2855,114.1577,Mini Hotel Central
9,Butaritari,KI,82.35,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
11,Kapaa,US,75.20,broken clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
12,Mar Del Plata,AR,71.01,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
13,Carnarvon,AU,66.20,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
15,San Juan,AR,78.53,broken clouds,-31.5375,-68.5364,Hotel Provincial
16,Batticaloa,LK,80.37,overcast clouds,7.7102,81.6924,Grand Star Guest House
20,Imphal,IN,67.50,broken clouds,24.8167,93.9500,State Guest House
22,Ocampo,MX,89.01,scattered clouds,21.6500,-101.5000,Hotel Estancia del Vaquero
23,Buraydah,SA,86.00,clear sky,26.3260,43.9750,فندق راحة السلام


In [39]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   339
Country                339
Max Temp               339
Current Description    339
Lat                    339
Lng                    339
Hotel Name             339
dtype: int64

In [43]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file)

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [46]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center = (30.0,31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))